# Paper 1: Cross-Architecture Knowledge Transfer via HDC
## Experiment 2: Teacher Size Study

Tests whether larger teacher models improve HDC transfer quality.

**Author**: Nikolay Yudin | **Project**: SEP | **Repo**: github.com/nick-yudin/SEP

In [ ]:
!pip install -q transformers datasets torch numpy scikit-learn matplotlib tqdm accelerate bitsandbytes

In [ ]:
# HF Login for Llama access
from huggingface_hub import login
try:
    from google.colab import userdata
    login(token=userdata.get('HF_TOKEN'))
except: pass

In [ ]:
import torch, torch.nn as nn, numpy as np, json, gc, os, shutil
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

In [ ]:
CONFIG = {
    'models': {
        'distilbert': {'name': 'distilbert-base-uncased', 'type': 'encoder', 'size': '66M'},
        'gpt2': {'name': 'gpt2', 'type': 'decoder', 'size': '124M'},
        'llama8b': {'name': 'meta-llama/Llama-3.1-8B', 'type': 'decoder', 'size': '8B'},
        'qwen14b': {'name': 'Qwen/Qwen2.5-14B', 'type': 'decoder', 'size': '14B'},
    },
    'train_size': 2000, 'test_size': 500, 'anchor_size': 500,
    'hdc_dim': 4096, 'tau': 0.3, 'seeds': [42, 123, 456],
    'contrastive_epochs': 20, 'use_4bit': True,
}
EXPERIMENTS = [('gpt2','distilbert'),('distilbert','distilbert'),('llama8b','distilbert'),('qwen14b','distilbert')]

In [ ]:
class EmbeddingExtractor:
    def __init__(self, model_key, config, device='cuda'):
        self.model_key = model_key
        info = config['models'][model_key]
        self.model_name, self.model_type = info['name'], info['type']
        print(f'Loading {model_key} ({info["size"]})...')
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None: self.tokenizer.pad_token = self.tokenizer.eos_token
        
        is_large = 'B' in info['size'] and int(info['size'].replace('B','')) >= 8
        if is_large and config.get('use_4bit'):
            print('  Using 4-bit quantization...')
            bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
            ModelClass = AutoModelForCausalLM if self.model_type == 'decoder' else AutoModel
            self.model = ModelClass.from_pretrained(self.model_name, quantization_config=bnb, device_map='auto', trust_remote_code=True)
        else:
            if self.model_type == 'decoder':
                self.model = AutoModelForCausalLM.from_pretrained(self.model_name, trust_remote_code=True).to(device)
            else:
                self.model = AutoModel.from_pretrained(self.model_name).to(device)
        self.model.eval()
        self.embed_dim = self.model.config.hidden_size
        self.device = device
        print(f'✅ {model_key}: {self.embed_dim}d')
    
    def encode(self, texts, batch_size=8):
        all_emb = []
        for i in tqdm(range(0, len(texts), batch_size), desc=f'Encoding {self.model_key}'):
            batch = texts[i:i+batch_size]
            inputs = self.tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors='pt')
            dev = self.model.device if hasattr(self.model, 'device') else self.device
            inputs = {k: v.to(dev) for k, v in inputs.items()}
            with torch.no_grad():
                if self.model_type == 'decoder':
                    hidden = self.model(**inputs, output_hidden_states=True).hidden_states[-1]
                else:
                    hidden = self.model(**inputs).last_hidden_state
                mask = inputs['attention_mask'].unsqueeze(-1)
                emb = (hidden * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
            all_emb.append(emb.float().cpu().numpy())
        return np.vstack(all_emb)
    
    def clear(self):
        del self.model, self.tokenizer
        torch.cuda.empty_cache(); gc.collect()
        cache = os.path.expanduser('~/.cache/huggingface/hub')
        if os.path.exists(cache):
            for f in os.listdir(cache):
                if any(x in f.lower() for x in ['llama','qwen']):
                    shutil.rmtree(os.path.join(cache,f), ignore_errors=True)
        print(f'  Cleared {self.model_key}')

In [ ]:
class HDCEncoder:
    def __init__(self, input_dim, hdc_dim, tau=0.3, seed=42):
        np.random.seed(seed)
        self.proj = np.random.randn(input_dim, hdc_dim).astype(np.float32)
        self.proj /= np.linalg.norm(self.proj, axis=0, keepdims=True)
        self.tau = tau
    def encode(self, emb):
        p = emb @ self.proj
        thr = self.tau * np.std(p, axis=1, keepdims=True)
        t = np.zeros_like(p, dtype=np.int8)
        t[p > thr] = 1; t[p < -thr] = -1
        return t

class ContrastiveAligner(nn.Module):
    def __init__(self, t_dim, s_dim, h_dim=512):
        super().__init__()
        self.t_proj = nn.Sequential(nn.Linear(t_dim, h_dim), nn.ReLU(), nn.Linear(h_dim, h_dim))
        self.s_proj = nn.Sequential(nn.Linear(s_dim, h_dim), nn.ReLU(), nn.Linear(h_dim, h_dim))
    def forward(self, t, s):
        return nn.functional.normalize(self.t_proj(t), dim=-1), nn.functional.normalize(self.s_proj(s), dim=-1)

def train_aligner(t_emb, s_emb, h_dim, epochs=20, device='cuda'):
    h_dim = max(h_dim, min(t_emb.shape[1], s_emb.shape[1]))
    print(f'  Aligner: teacher_dim={t_emb.shape[1]}, student_dim={s_emb.shape[1]}, hidden_dim={h_dim}')
    aligner = ContrastiveAligner(t_emb.shape[1], s_emb.shape[1], h_dim).to(device)
    opt = torch.optim.Adam(aligner.parameters(), lr=1e-3)
    T, S = torch.tensor(t_emb, dtype=torch.float32).to(device), torch.tensor(s_emb, dtype=torch.float32).to(device)
    bs = min(256, len(T))
    for _ in range(epochs):
        perm = torch.randperm(len(T))
        for i in range(0, len(T), bs):
            idx = perm[i:i+bs]
            tp, sp = aligner(T[idx], S[idx])
            pos = (tp * sp).sum(-1)
            neg = (tp * sp[torch.roll(torch.arange(len(idx)), 1)]).sum(-1)
            loss = -pos.mean() + torch.clamp(neg + 0.5, min=0).mean()
            opt.zero_grad(); loss.backward(); opt.step()
    aligner.eval()
    with torch.no_grad(): tp, sp = aligner(T, S); sim = (tp * sp).sum(-1).mean().item()
    return aligner, sim

In [ ]:
def load_data(seed=42):
    np.random.seed(seed)
    ds = load_dataset('glue', 'sst2')['train']
    idx = np.random.permutation(len(ds))[:CONFIG['train_size']+CONFIG['test_size']+CONFIG['anchor_size']]
    texts = [ds['sentence'][i] for i in idx]
    labels = [ds['label'][i] for i in idx]
    n1, n2 = CONFIG['train_size'], CONFIG['train_size']+CONFIG['test_size']
    return {'train_texts': texts[:n1], 'train_labels': labels[:n1],
            'test_texts': texts[n1:n2], 'test_labels': labels[n1:n2],
            'anchor_texts': texts[n2:]}

data = load_data()
print(f'Data: {len(data["train_texts"])} train, {len(data["test_texts"])} test')

In [ ]:
def run_experiment(teacher_key, student_key, data, seed=42):
    np.random.seed(seed); torch.manual_seed(seed)
    print(f'\n--- {teacher_key} → {student_key} (seed={seed}) ---')
    result = {'teacher': teacher_key, 'student': student_key, 'seed': seed}
    try:
        # Teacher
        t_ext = EmbeddingExtractor(teacher_key, CONFIG, device)
        t_train = t_ext.encode(data['train_texts'])
        t_anchor = t_ext.encode(data['anchor_texts'])
        result['teacher_dim'] = t_ext.embed_dim
        t_ext.clear()
        
        # Student
        s_ext = EmbeddingExtractor(student_key, CONFIG, device)
        s_train = s_ext.encode(data['train_texts'])
        s_test = s_ext.encode(data['test_texts'])
        s_anchor = s_ext.encode(data['anchor_texts'])
        s_ext.clear()
        
        # Ceiling
        print('  Computing student ceiling...')
        hdc = HDCEncoder(s_train.shape[1], CONFIG['hdc_dim'], CONFIG['tau'], seed)
        clf = LogisticRegression(max_iter=1000, random_state=seed)
        clf.fit(hdc.encode(s_train), data['train_labels'])
        ceiling = clf.score(hdc.encode(s_test), data['test_labels'])
        result['student_ceiling'] = ceiling
        print(f'  Student ceiling: {ceiling:.1%}')
        
        # Alignment
        print('  Training alignment...')
        h_dim = max(768, min(t_train.shape[1], s_train.shape[1]) // 2)
        aligner, sim = train_aligner(t_anchor, s_anchor, h_dim, CONFIG['contrastive_epochs'], device)
        result['alignment_similarity'] = sim
        print(f'  Alignment similarity: {sim:.3f}')
        
        # Transfer
        aligner.eval()
        with torch.no_grad():
            T = torch.tensor(t_train, dtype=torch.float32).to(device)
            S = torch.tensor(s_test, dtype=torch.float32).to(device)
            t_aligned = nn.functional.normalize(aligner.t_proj(T), dim=-1).cpu().numpy()
            s_aligned = nn.functional.normalize(aligner.s_proj(S), dim=-1).cpu().numpy()
        
        hdc2 = HDCEncoder(t_aligned.shape[1], CONFIG['hdc_dim'], CONFIG['tau'], seed)
        clf2 = LogisticRegression(max_iter=1000, random_state=seed)
        clf2.fit(hdc2.encode(t_aligned), data['train_labels'])
        acc = clf2.score(hdc2.encode(s_aligned), data['test_labels'])
        
        result['transfer_accuracy'] = acc
        result['transfer_efficiency'] = acc / ceiling
        print(f'  Transfer accuracy: {acc:.1%}')
        print(f'  Transfer efficiency: {result["transfer_efficiency"]:.1%}')
    except Exception as e:
        print(f'  ERROR: {e}')
        result['error'] = str(e)
    return result

In [ ]:
print('=' * 60)
print('RUNNING EXPERIMENTS')
print('=' * 60)

all_results = []
for t_key, s_key in EXPERIMENTS:
    print(f'\n{"#"*60}\n# {t_key.upper()} → {s_key.upper()}\n{"#"*60}')
    for seed in CONFIG['seeds']:
        all_results.append(run_experiment(t_key, s_key, data, seed))

print('\n' + '=' * 60)
print('ALL EXPERIMENTS COMPLETE')
print('=' * 60)

In [ ]:
import pandas as pd
df = pd.DataFrame([r for r in all_results if 'error' not in r])

print('\nSUMMARY BY TEACHER:')
for t in df['teacher'].unique():
    sub = df[df['teacher'] == t]
    print(f"  {t}: acc={sub['transfer_accuracy'].mean():.1%} ± {sub['transfer_accuracy'].std():.1%}, eff={sub['transfer_efficiency'].mean():.1%}")

In [ ]:
# Save
def serialize(obj):
    if isinstance(obj, dict): return {k: serialize(v) for k, v in obj.items()}
    if isinstance(obj, list): return [serialize(v) for v in obj]
    if isinstance(obj, (np.floating, np.integer)): return float(obj)
    return obj

with open('paper1_experiment2_results.json', 'w') as f:
    json.dump({'config': CONFIG, 'experiments': serialize(all_results)}, f, indent=2)
print('Saved to paper1_experiment2_results.json')